Import the required libraries

In [2]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from apsimNGpy.senstivity.sensitivity import ConfigProblem, run_sensitivity

Define APSIM model inputs as follows

In [7]:
params = {
       ".Simulations.Simulation.Field.Sow using a variable rule?Population": (2, 10),
       ".Simulations.Simulation.Field.Fertilise at sowing?Amount": (0, 300),
       ".Simulations.Simulation.Field.Maize.CultivarFolder.Dekalb_XL82?[Leaf].Photosynthesis.RUE.FixedValue": (
           1.2, 2.2),
   }

The base parameter path and the specific parameter name or sub-path are separated using one of the following delimiters: ?, ::, or @.

The values associated with each parameter define the lower and upper bounds of the sampling space used during sensitivity analysis.

Initialize the problem as follows:

In [4]:
runner = ConfigProblem(
    base_model="Maize",
    params=params,
    outputs=["Yield", "Maize.AboveGround.N"],
)

SALib provides an object-oriented interface that simplifies the sensitivity analysis triad of sampling, evaluation, and analysis. In apsimNGpy, this workflow is further streamlined by wrapping all three steps into a single method, run_sensitivity().



In [ ]:
Si_sobol = run_sensitivity(
    runner,
    method="sobol",
    N=2 ** 6,  # ← base sample size should be power of 2
    engine = 'csharp',
    sample_options={
        "calc_second_order": False,
        "skip_values": 1024,
         "seed": 42,
    },
    analyze_options={
        "conf_level": 0.95,
        "num_resamples": 1000,
        "print_to_console": True,
        "calc_second_order": False,
    },
)

Processing 256 jobs wait.. ██████▋     67%  >> completed (elapsed=>03:38, eta=>01:50) 

The returned object is an instance of the SALib.util.problem.ProblemSpec class. When evaluated in the Python console, this object displays a summary of the problem definition and the computed sensitivity indices, as shown below.

Converting outputs to pandas

In [ ]:
Si_sobol.to_df()

Key important attributes for samples, model results and analyses extraction available on the returned instance are shown below.

In [ ]:
print( Si_sobol.samples)
print( Si_sobol.results)
print( Si_sobol.analysis)

Attribute ``apsim_results`` allows you to extract the evaluation or APSIM simulated results

In [ ]:
Si_sobol.apsim_results